#

In [45]:
import Pkg;
Pkg.add(["AlgebraOfGraphics", "CairoMakie", "DataFrames", "Arrow", "MakiePublication"])

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`

In [46]:
using AlgebraOfGraphics
using AlgebraOfGraphics: density
using CairoMakie
using DataFrames
using Arrow

set_aog_theme!()

In [47]:
function load(path :: String)
    df = path |> Arrow.Table |> DataFrame |> dropmissing

    # Iterate through each column in the DataFrame

    filter(row -> all(x -> !(x isa Number && isnan(x)), row), df)
end

begin
    path01 = "../data/psp_ids_dataset.arrow"
    path02 = "../data/psp_ids_dataset_r.arrow"
    path03 = "../data/wind_ids_dataset_r.arrow"

    psp_df = load(path01)
    psp_df_r = load(path02)
    wind_df=  load(path03)
end
# psp_df = Dataset("../data/psp_ids_dataset.parquet")

In [48]:
function process!(df)
    df[!, "n.mean"] = (df[!, "n.before"] .+ df[!, "n.after"]) ./ 2
    df[!, "B.mean"] = (df[!, "B.before"] .+ df[!, "B.after"]) ./ 2
    df[!, "T.mean"] = (df[!, "T.before"] .+ df[!, "T.after"]) ./ 2
end 

begin
    # concat dataframe with add column dataset to identify the dataset
    psp_df[!, :dataset] .= "PSP (High Res)"
    psp_df_r[!, :dataset] .= "PSP (Low Res)"
    wind_df[!, :dataset] .= "Wind"

    process!(psp_df)
    process!(psp_df_r)
    process!(wind_df)

    df = vcat(psp_df, psp_df_r, wind_df, cols=:intersect)
end


In [49]:
begin
    j_label = L"Current Intensity ($nA/m^2$)"
    j_norm_label = L"Normalized Current Intensity ($I_A$)"
    l_label = L"Thickness ($km$)"
    l_norm_label = L"Normalized Thickness ($d_i$)"

    thickness_map = (l -> abs(l)) => l_label
    thickness_norm_map = (l -> abs(l)) => l_norm_label
    current_map = (j -> abs(j)) => j_label
    current_norm_map = (j -> abs(j)) => j_norm_label

    data_layer_a = (data(psp_df) + data(psp_df_r) * visual(alpha=0.6) + data(wind_df) * visual(alpha=0.3)) * mapping(color=:dataset, marker=:dataset)
    plt = data_layer_a * visual(Scatter)

    fig = Figure(size = (1000, 500))

    axis = (yscale=log10, xscale=log10)
    grid = draw!(fig[2, 1], plt * mapping(:L_k => thickness_map, :j0_k => current_map), axis=axis)
    draw!(fig[2, 2], plt * mapping(:L_k_norm => thickness_norm_map, :j0_k_norm => current_norm_map), axis=axis)

    legend!(fig[1, 1:2], grid, titleposition=:left, orientation=:horizontal)

    # save("../figures/thickness_current.png", fig,  px_per_unit = 4)
    # save("../figures/thickness_current.pdf", fig)

    fig
end


In [50]:
# plot the density distribution of the thickness and current density
begin

    data_layer = data(df) * mapping(color=:dataset, marker=:dataset) 
    plt = data_layer * density()

    fig = Figure(size = (1000, 700))
    mapping_layer_j = mapping(:j0_k => (j -> log10(abs(j))) => L"Log %$(j_label)")
    mapping_layer_j_norm = mapping(:j0_k_norm => (j -> log10(abs(j))) => L"Log %$(j_norm_label)")

    mapping_layer_l = mapping(:L_k => (l -> log10(abs(l))) => L"Log %$(l_label)")
    mapping_layer_l_norm = mapping(:L_k_norm => (l -> log10(abs(l))) => L"Log %$(l_norm_label)")

    grid = draw!(fig[2, 1], plt * mapping_layer_j)
    draw!(fig[2, 2], plt * mapping_layer_j_norm)
    draw!(fig[3, 1], plt * mapping_layer_l)
    draw!(fig[3, 2], plt * mapping_layer_l_norm)
    
    legend!(fig[1, 1:2], grid, titleposition=:left, orientation=:horizontal)
    # legend!(fig[1,3], grid,)

    save("../figures/density_distribution.pdf", fig)
    save("../figures/density_distribution.png", fig, px_per_unit = 4)
    fig

end

In [51]:
begin
    # Create a density plot of the data

    dn_over_n = ("n.change", "n.mean") => (/) => L"\Delta n/n"
    dB_over_B = ("B.change", "b_mag") => (/) => L"\Delta B/B"
    dT_over_T = ("T.change", "T.mean") => (/) => L"\Delta T/T"
    
    plt = data_layer_a * mapping(dn_over_n, dB_over_B) * (linear() + mapping())

    fig = Figure(size = (1000, 500))
    grid = draw!(fig[2, 1], plt)
    draw!(fig[2, 2], plt, axis=(; limits=((-0.5,0.5), (-0.5, 0.5))))
    legend!(fig[1, 1:2], grid, titleposition=:left, orientation=:horizontal)

    fig
end

In [52]:
begin
    plt = data_layer_a * mapping(dT_over_T, dn_over_n) * (linear() + mapping())

    fig = Figure(size = (1000, 500))
    grid = draw!(fig[2, 1], plt)
    draw!(fig[2, 2], plt, axis=(; limits=((-0.5,0.5), (-0.5, 0.5))))
    legend!(fig[1, 1:2], grid, titleposition=:left, orientation=:horizontal)

    fig
end

In [53]:
begin
    plt = data_layer_a * mapping(dT_over_T, db_over_b) * (linear() + mapping())

    fig = Figure(size = (1000, 500))
    grid = draw!(fig[2, 1], plt)
    draw!(fig[2, 2], plt, axis=(; limits=((-0.5,0.5), (-0.5, 0.5))))
    legend!(fig[1, 1:2], grid, titleposition=:left, orientation=:horizontal)

    fig
end

In [54]:
log_axis = (; yscale=log10, xscale=log10)

(yscale = log10, xscale = log10)

In [56]:
v_Alven_map = "v.Alfven.change" => abs => L"Δ V_{A}"
v_ion_map = "v.ion.change" => abs => L"Δ V_{i}"

plt = data_layer_a * mapping(v_Alven_map, v_ion_map) * (linear() + mapping())

fig = Figure(size=(1000, 500))
limit_axis = (; limits=((1, 200), (1, 200)))
axis = merge(log_axis, limit_axis)
grid = draw!(fig[2, 1], plt)
draw!(fig[2, 2], plt, axis=axis)
legend!(fig[1, 1:2], grid, titleposition=:left, orientation=:horizontal)

fig

In [70]:
plt = data_layer_a * mapping("v.Alfven.change.l" => abs => L"Δ V_{A,l}", "v.ion.change.l" => abs => L"\Delta V_{i,l}") * (linear() + mapping())

fig = Figure(size = (1000, 500))
limit_axis = (; limits=((2, 400), (2, 400)))

grid = draw!(fig[2, 1], plt)
draw!(fig[2, 2], plt, axis=merge(log_axis, limit_axis))
legend!(fig[1, 1:2], grid, titleposition=:left, orientation=:horizontal)

fig